# Promethee Veto
This module shows an example of how to use the "Promethee Veto" module

## Definition of inputs and problem formalization

In [6]:
import pandas as pd
from core.enums import Direction, GeneralCriterion
from modular_parts.preference import compute_preference_indices, compute_veto


alternatives = pd.Index([f"a{i}" for i in range(1, 6)])
criteria = pd.Index([f"g{i}" for i in range(1, 6)])
profiles = pd.Index([f"r{i}" for i in range(1, 4)])
criteria_directions = pd.Series([Direction.MAX, Direction.MAX, Direction.MIN, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.25, 0.2, 0.1, 0.15, 0.3], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.USUAL,
                        GeneralCriterion.U_SHAPE,
                        GeneralCriterion.LEVEL,
                        GeneralCriterion.V_SHAPE_INDIFFERENCE,
                        GeneralCriterion.GAUSSIAN], index=criteria)
preference_thresholds = pd.Series([None, None, 14, 70, None], index=criteria)
indifference_thresholds = pd.Series([None, 30, 7, 30, None], index=criteria)
standard_deviations = pd.Series([None, None, None, None, 3], index=criteria)

veto_thresholds = pd.Series([10, 80, 28, 150, None], index=criteria)

alternatives_performances = pd.DataFrame([[10, 113, 70, 556, 7],
                                          [12, 153, 49, 447, 8],
                                          [15, 199, 62, 589, 14],
                                          [19, 185, 41, 606, 11],
                                          [14, 146, 55, 495, 8],], index=alternatives, columns=criteria)
profiles_performances = pd.DataFrame([[11, 130, 65, 600, 7],
                                      [14, 150, 55, 525, 9],
                                      [17, 180, 45, 450, 11]], index=profiles, columns=criteria)


## Required preferences

### Alternatives vs alternatives preferences

In [7]:
aa_preference_indices, _ = compute_preference_indices(alternatives_performances,
                                                      preference_thresholds,
                                                      indifference_thresholds,
                                                      standard_deviations,
                                                      generalised_criteria,
                                                      criteria_directions,
                                                      criteria_weights)

In [8]:
aa_preference_indices

,a1,a2,a3,a4,a5
a1,0.000,0.000,0.011,0.075,0.000
a2,0.716,0.000,0.200,0.150,0.068
a3,0.780,0.709,0.000,0.118,0.709
a4,0.727,0.618,0.350,0.000,0.618
a5,0.682,0.250,0.150,0.150,0.000


### Alternatives vs profiles preferences

In [10]:
ap_preference_indices, _ = compute_preference_indices(alternatives_performances,
                                                      preference_thresholds,
                                                      indifference_thresholds,
                                                      standard_deviations,
                                                      generalised_criteria,
                                                      criteria_directions,
                                                      criteria_weights,
                                                      profiles_performances)

In [12]:
ap_preference_indices[0]

,r1,r2,r3
a1,0.052,0.000,0.000
a2,0.516,0.150,0.000
a3,0.730,0.675,0.118
a4,0.727,0.560,0.250
a5,0.466,0.000,0.000


In [13]:
ap_preference_indices[1]

,a1,a2,a3,a4,a5
r1,0.250,0.000,0.000,0.00,0.000
r2,0.614,0.266,0.128,0.15,0.016
r3,0.877,0.368,0.500,0.15,0.674


## Usage of Promethee Veto

### Alternatives vs alternatives preferences

### Veto only

In [14]:
aa_veto_indices, aa_partial_veto_indices = compute_veto(alternatives_performances,
                                                        criteria_weights,
                                                        veto_thresholds,
                                                        criteria_directions)

In [15]:
aa_partial_veto_indices

a1  a2  a3  a4  a5
criteria                       
g1       a1   0   0   0   0   0
         a2   0   0   0   0   0
         a3   0   0   0   0   0
         a4   0   0   0   0   0
         a5   0   0   0   0   0
g2       a1   0   0   1   0   0
         a2   0   0   0   0   0
         a3   0   0   0   0   0
         a4   0   0   0   0   0
         a5   0   0   0   0   0
g3       a1   0   0   0   1   0
         a2   0   0   0   0   0
         a3   0   0   0   0   0
         a4   0   0   0   0   0
         a5   0   0   0   0   0
g4       a1   0   0   0   0   0
         a2   0   0   0   0   0
         a3   0   0   0   0   0
         a4   0   1   0   0   0
         a5   0   0   0   0   0
g5       a1   0   0   0   0   0
         a2   0   0   0   0   0
         a3   0   0   0   0   0
         a4   0   0   0   0   0
         a5   0   0   0   0   0

As you can see, the veto occurred f.e. on the **g2** criterion for alternative **a1** over alternative **a3**.

In [16]:
aa_veto_indices

,a1,a2,a3,a4,a5
a1,0,0,1,1,0
a2,0,0,0,0,0
a3,0,0,0,0,0
a4,0,1,0,0,0
a5,0,0,0,0,0


#### Veto applied on preferences

In [17]:
aa_veto_indices, aa_partial_veto_indices, final_preference = compute_veto(alternatives_performances,
                                                                          criteria_weights,
                                                                          veto_thresholds,
                                                                          criteria_directions,
                                                                          preferences=aa_preference_indices)

In [18]:
final_preference

,a1,a2,a3,a4,a5
a1,0.000,0.000,0.00,0.000,0.000
a2,0.716,0.000,0.20,0.150,0.068
a3,0.780,0.709,0.00,0.118,0.709
a4,0.727,0.000,0.35,0.000,0.618
a5,0.682,0.250,0.15,0.150,0.000


In the relations in which the veto occurred, the preference is set to 0.

### Alternatives vs profiles

#### Veto only

In [22]:
ap_veto_indices, ap_partial_veto_indices = compute_veto(alternatives_performances,
                                                        criteria_weights,
                                                        veto_thresholds,
                                                        criteria_directions,
                                                        profiles_performance=profiles_performances)

In [24]:
ap_partial_veto_indices[0]

r1  r2  r3
criteria               
g1       a1   0   0   0
         a2   0   0   0
         a3   0   0   0
         a4   0   0   0
         a5   0   0   0
g2       a1   0   0   0
         a2   0   0   0
         a3   0   0   0
         a4   0   0   0
         a5   0   0   0
g3       a1   0   0   0
         a2   0   0   0
         a3   0   0   0
         a4   0   0   0
         a5   0   0   0
g4       a1   0   0   0
         a2   0   0   0
         a3   0   0   0
         a4   0   0   1
         a5   0   0   0
g5       a1   0   0   0
         a2   0   0   0
         a3   0   0   0
         a4   0   0   0
         a5   0   0   0

In [25]:
ap_partial_veto_indices[1]

a1  a2  a3  a4  a5
criteria                       
g1       r1   0   0   0   0   0
         r2   0   0   0   0   0
         r3   0   0   0   0   0
g2       r1   0   0   0   0   0
         r2   0   0   0   0   0
         r3   0   0   0   0   0
g3       r1   0   0   0   0   0
         r2   0   0   0   0   0
         r3   0   0   0   0   0
g4       r1   0   1   0   0   0
         r2   0   0   0   0   0
         r3   0   0   0   0   0
g5       r1   0   0   0   0   0
         r2   0   0   0   0   0
         r3   0   0   0   0   0

### Veto applied on preferences

In [28]:
ap_veto_indices, ap_partial_veto_indices, final_preference = compute_veto(alternatives_performances,
                                                                          criteria_weights,
                                                                          veto_thresholds,
                                                                          criteria_directions,
                                                                          profiles_performance=profiles_performances,
                                                                          preferences=ap_preference_indices)

In [29]:
final_preference[0]

,r1,r2,r3
a1,0.052,0.000,0.000
a2,0.516,0.150,0.000
a3,0.730,0.675,0.118
a4,0.727,0.560,0.000
a5,0.466,0.000,0.000


In [30]:
final_preference[1]

,a1,a2,a3,a4,a5
r1,0.250,0.000,0.000,0.00,0.000
r2,0.614,0.266,0.128,0.15,0.016
r3,0.877,0.368,0.500,0.15,0.674
